In [1]:
import os
from fnmatch import fnmatch
from trainData_featExt_glcm import trainData_featExt_glcm
from trainData_featExt_gabor import trainData_featExt_gabor
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.lbp import Lbp
from sklearn.neighbors import NearestNeighbors
import sklearn
import time
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize
from sklearn import preprocessing

In [2]:
#Feature_matrix_training = genfromtxt('Featurevector-train')
Feature_matrix_training = np.load("train_make_classification.npy")

In [3]:
validation_feature_matrix = np.load("validataion_make_clssification_data.npy")

In [4]:
#Train a KNN model using training data
t0 = time.time()
knn_model = NearestNeighbors(500, algorithm = 'kd_tree', metric ='euclidean')
knn_model.fit(Feature_matrix_training) 
print("traing time: ", time.time() - t0)

('traing time: ', 0.02747321128845215)


In [5]:
#find k nearest points for each sample
kth_dist, kth_ind = knn_model.kneighbors(Feature_matrix_training)
#1-D array contains distances of each data point to its kth nearest point 
kth_nearest_dist = kth_dist[:,-1]

In [41]:
#find k nearest points for each valid sample
kth_dist2, kth_ind2 = knn_model.kneighbors(validation_feature_matrix)
#1-D array contains distances of each data point to its kth nearest point
kth_nearest_dist2 = kth_dist2[:,-1]
labels = np.repeat(0,10000)
#each validation data point whose distance to its kth nearest exceeds the 
#threshold, which is np.amax(kth_nearest_dist) is novel (1)
labels[kth_nearest_dist2 > np.max(kth_nearest_dist)] = 1

In [42]:
#Ground truth
y1 = np.repeat(0, 6000) #normal
y2 = np.repeat(1, 4000) #abnormal
y = np.concatenate((y1,y2))

In [43]:
f1 = sklearn.metrics.f1_score(y, labels, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, kth_nearest_dist2)
print ("f1: ", f1 , "auc: ", auc)

('f1: ', 0.99906214751400479, 'auc: ', 0.99999962500000006)
